In [ ]:
import os 

# Please specify the following path to WaterMAI_dataset directory
path = "your_path_to/WaterMAI_dataset"
os.chdir(path)

In [3]:
import numpy as np
import shutil
import cv2
import tifffile as tiff

In [4]:
# NDWI
def NDWI(G, NIR):
    """Compute the NDWI
    INPUT : G (np.array) -> the Green band images as a numpy array of float
            NIR (np.array) -> the Near Infrared images as a numpy array of float
    OUTPUT : NDWI (np.array) -> the NDWI
    """
    NDWI = (G - NIR) / (G + NIR + 1e-12)
    return NDWI

def GreenNirNDWI(greennir):
    """ Input
            greennir _ contains only Green and NIR bands 
    """
    # compute NDWI (G, NIR)
    ndwi = NDWI(greennir[:, :, 0], greennir[:, :, 1])
    ndwi = np.round(ndwi).astype(np.uint8)

    # define images to further process : R, NIR, NDWI
    img_to_process = [greennir[:, :, 0], greennir[:, :, 1], ndwi]

    return np.stack(img_to_process, axis=2)

## Generate 'Green + NIR + NDWI' (3 bands) images & masks
- This cell generate 'Green + NIR + NDWI' (3 bands) images from the **images/co** and **images/ir** folders & saves them in **images/gNirNdwi** folder
- This cell also generates a label mask named **labels/masks_gnirndwi** for 'Green + NIR + NDWI' images. This label mask is similar to the **labels/masks_co** label mask, with the only difference being the name of each respective folder

In [14]:
# (Green, NIR, NDWI)
rgb_dir = './images/co'
nir_dir = './images/ir'

gnirndwi_dir = './images/gnirndwi'
if not os.path.exists(gnirndwi_dir):
    os.makedirs(gnirndwi_dir)

# IMAGES
file_names = [jpg_f.split('.')[0][:-3] for jpg_f in os.listdir(rgb_dir) if jpg_f.endswith('.jpg')]

for name in file_names:
    # Full path rgb, nir
    rgb_path = os.path.join(rgb_dir, name + '_co.jpg')
    nir_path = os.path.join(nir_dir, name + '_ir.jpg')
    gnirndwi_path = os.path.join(gnirndwi_dir, name + '_gnirndwi.jpg')
    
    # Read the images
    rgb_img = cv2.imread(rgb_path)
    rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
    nir_img = cv2.imread(nir_path)
    
    # Construct combination (G, NIR, NDWI)
    green = rgb_img[:, :, 1]
    nir = nir_img[:, :, 0]
    gnirndwi_img = GreenNirNDWI(np.stack([green, nir], axis=2))
    
    # Save the image as a .jpg file
    cv2.imwrite(gnirndwi_path, gnirndwi_img)

# Generate train_gnirndwi.txt and val_gnirndwi.txt metadata files & save in images/gnirndwi folder
txt_files = [txt_f.split('/')[-1].split('.')[0] for txt_f in os.listdir(rgb_dir) if txt_f.endswith('.txt')]

for txt_file in txt_files:
    # Full path txt
    txt_name = txt_file.replace('_co', '_gnirndwi')
    txt_gnirndwi = os.path.join(gnirndwi_dir, txt_name + '.txt')
    txt_co = os.path.join(rgb_dir, txt_file + '.txt')
    
    # Read the file
    with open(txt_co, 'r') as file:
        lines = file.readlines()

    # Modify the paths
    modified_lines = []
    for line in lines:
        modified_line = line.replace('_co', '_gnirndwi')
        modified_lines.append(modified_line)

    # Write the modified paths back to the file
    with open(txt_gnirndwi, 'w') as file:
        file.writelines(modified_lines)

# LABELS
co_label = './labels/masks_co'
gnirndwi_label = './labels/masks_gnirndwi'
if not os.path.exists(gnirndwi_label):
    os.makedirs(gnirndwi_label)

for source_name in os.listdir(co_label):
    if source_name.endswith('.png'):
        # Modify the file name as needed
        new_name = source_name.replace('_co', '_gnirndwi')
        shutil.copy(os.path.join(co_label, source_name), os.path.join(gnirndwi_label, new_name))

/tmp/ipykernel_23729/1353729320.py:17: RuntimeWarning: invalid value encountered in cast
  ndwi = np.round(ndwi).astype(np.uint8)


## Generate 'RGB + NIR' (4 bands) images & masks
- This cell generate 'RGB + NIR' (4 bands) images from the **images/co** and **images/ir** folders & saves them in **images/coir** folder
- This cell generates a label mask named **labels/masks_coir** for 'RGB + NIR' images. This label mask is similar to the **labels/masks_co** label mask, with the only difference being the name of each respective folder

In [15]:
# RGB + NIR _ 4 bands
rgb_dir = './images/co'
nir_dir = './images/ir'

coir_dir = './images/coir'
if not os.path.exists(coir_dir):
    os.makedirs(coir_dir)

# IMAGES
file_names = [jpg_f.split('.')[0][:-3] for jpg_f in os.listdir(rgb_dir) if jpg_f.endswith('.jpg')]

for name in file_names:
    # Full path rgb, nir
    rgb_path = os.path.join(rgb_dir, name + '_co.jpg')
    nir_path = os.path.join(nir_dir, name + '_ir.jpg')
    coir_path = os.path.join(coir_dir, name + '_coir.tiff')
    
    # Read the images
    rgb_img = cv2.imread(rgb_path)
    rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
    nir_band = cv2.imread(nir_path)[:, :, 0]    

    # Stack the arrays along the third axis (4 bands)
    combined_arr = np.stack([rgb_img[:, :, 0], rgb_img[:, :, 1], rgb_img[:, :, 2], nir_band], axis=2)

    # Save the new image as a .tiff file
    tiff.imwrite(coir_path, combined_arr)

# Generate train_coir.txt and val_coir.txt metadata files & save in images/coir folder
txt_files = [txt_f.split('/')[-1].split('.')[0] for txt_f in os.listdir(rgb_dir) if txt_f.endswith('.txt')]

for txt_file in txt_files:
    # Full path txt
    txt_name = txt_file.replace('_co', '_coir')
    txt_coir = os.path.join(coir_dir, txt_name + '.txt')
    txt_co = os.path.join(rgb_dir, txt_file + '.txt')
    
    # Read the file
    with open(txt_co, 'r') as file:
        lines = file.readlines()

    # Modify the paths
    modified_lines = []
    for line in lines:
        modified_line = line.replace('_co', '_coir')
        modified_lines.append(modified_line)

    # Write the modified paths back to the file
    with open(txt_coir, 'w') as file:
        file.writelines(modified_lines)

# LABELS
co_label = './labels/masks_co'
coir_label = './labels/masks_coir'
if not os.path.exists(coir_label):
    os.makedirs(coir_label)

for source_name in os.listdir(co_label):
    if source_name.endswith('.png'):
        # Modify the file name as needed
        new_name = source_name.replace('_co', '_coir')
        shutil.copy(os.path.join(co_label, source_name), os.path.join(coir_label, new_name))

## Generate 'RGB + NDWI' (4 bands) images & masks
- This cell generate 'RGB + NDWI' (4 bands) images from the **images/co** and **images/ir** folders & saves them in **images/condwi** folder
- This cell generates a label mask named **labels/masks_condwi** for 'RGB + NDWI' images. This label mask is similar to the **labels/masks_co** label mask, with the only difference being the name of each respective folder

In [ ]:
# RGB + NDWI _ 4 bands
rgb_dir = './images/co'
nir_dir = './images/ir'

condwi_dir = './images/condwi'
if not os.path.exists(condwi_dir):
    os.makedirs(condwi_dir)

# IMAGES
file_names = [jpg_f.split('.')[0][:-3] for jpg_f in os.listdir(rgb_dir) if jpg_f.endswith('.jpg')]

for name in file_names:
    # Full path rgb, nir
    rgb_path = os.path.join(rgb_dir, name + '_co.jpg')
    nir_path = os.path.join(nir_dir, name + '_ir.jpg')
    condwi_path = os.path.join(condwi_dir, name + '_condwi.tiff')
    
    # Read the images
    rgb_img = cv2.imread(rgb_path)
    rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
    nir = cv2.imread(nir_path)[:, :, 0]
    green = rgb_img[:, :, 1]
    ndwi = NDWI(green, nir)
    ndwi = np.round(ndwi).astype(np.uint8)

    # Stack the arrays along the third axis (4 bands)
    combined_arr = np.stack([rgb_img[:, :, 0], rgb_img[:, :, 1], rgb_img[:, :, 2], ndwi], axis=2)

    # Save the new image as a .tiff file
    tiff.imwrite(condwi_path, combined_arr)

# Generate train_condwi.txt and val_condwi.txt metadata files & save in images/condwi folder
txt_files = [txt_f.split('/')[-1].split('.')[0] for txt_f in os.listdir(rgb_dir) if txt_f.endswith('.txt')]

for txt_file in txt_files:
    # Full path txt
    txt_name = txt_file.replace('_co', '_condwi')
    txt_condwi = os.path.join(condwi_dir, txt_name + '.txt')
    txt_co = os.path.join(rgb_dir, txt_file + '.txt')
    
    # Read the file
    with open(txt_co, 'r') as file:
        lines = file.readlines()

    # Modify the paths
    modified_lines = []
    for line in lines:
        modified_line = line.replace('_co', '_condwi')
        modified_lines.append(modified_line)

    # Write the modified paths back to the file
    with open(txt_condwi, 'w') as file:
        file.writelines(modified_lines)

# LABELS
co_label = './labels/masks_co'
condwi_label = './labels/masks_condwi'
if not os.path.exists(condwi_label):
    os.makedirs(condwi_label)

# Change the endpoint of the file name as needed
for source_name in os.listdir(co_label):
    if source_name.endswith('.png'):
        # Modify the file name as needed
        new_name = source_name.replace('_co', '_condwi')
        shutil.copy(os.path.join(co_label, source_name), os.path.join(condwi_label, new_name))

/tmp/ipykernel_23729/2184406245.py:24: RuntimeWarning: invalid value encountered in cast
  ndwi = np.round(ndwi).astype(np.uint8)


## Generate 'Red + NDWI + Blue' (3 bands) images & masks
- This cell generate 'Red + NDWI + Blue' (3 bands) images from the **images/co** and **images/ir** folders & saves them in **images/rndwib** folder
- This cell generates a label mask named **labels/masks_rndwib** for 'Red + NDWI + Blue' images. This label mask is similar to the **labels/masks_rndwib** label mask, with the only difference being the name of each respective folder

In [17]:
# Red + NDWI + Blue _ 3 bands
rgb_dir = './images/co'
nir_dir = './images/ir'

rndwib_dir = './images/rndwib'
if not os.path.exists(rndwib_dir):
    os.makedirs(rndwib_dir)

# IMAGES
file_names = [jpg_f.split('.')[0][:-3] for jpg_f in os.listdir(rgb_dir) if jpg_f.endswith('.jpg')]

for name in file_names:
    # Full path rgb, nir
    rgb_path = os.path.join(rgb_dir, name + '_co.jpg')
    nir_path = os.path.join(nir_dir, name + '_ir.jpg')
    rndwib_path = os.path.join(rndwib_dir, name + '_rndwib.tiff')
    
    # Read the images
    rgb_img = cv2.imread(rgb_path)
    rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
    nir = cv2.imread(nir_path)[:, :, 0]
    red = rgb_img[:, :, 0]
    green = rgb_img[:, :, 1]
    blue = rgb_img[:, :, 2]
    ndwi = NDWI(green, nir)
    ndwi = np.round(ndwi).astype(np.uint8)

    # Stack the arrays along the third axis (rndwib 3 bands)
    combined_arr = np.stack([rgb_img[:, :, 0], rgb_img[:, :, 1], rgb_img[:, :, 2], ndwi], axis=2)

    # Save the new image as a .tiff file
    tiff.imwrite(rndwib_path, combined_arr)

# Generate train_rndwib.txt and val_rndwib.txt metadata files & save in images/rndwib folder
txt_files = [txt_f.split('/')[-1].split('.')[0] for txt_f in os.listdir(rgb_dir) if txt_f.endswith('.txt')]

for txt_file in txt_files:
    # Full path txt
    txt_name = txt_file.replace('_co', '_rndwib')
    txt_rndwib = os.path.join(rndwib_dir, txt_name + '.txt')
    txt_co = os.path.join(rgb_dir, txt_file + '.txt')
    
    # Read the file
    with open(txt_co, 'r') as file:
        lines = file.readlines()

    # Modify the paths
    modified_lines = []
    for line in lines:
        modified_line = line.replace('_co', '_rndwib')
        modified_lines.append(modified_line)

    # Write the modified paths back to the file
    with open(txt_rndwib, 'w') as file:
        file.writelines(modified_lines)

# LABELS
co_label = './labels/masks_co'
rndwib_label = './labels/masks_rndwib'
if not os.path.exists(rndwib_label):
    os.makedirs(rndwib_label)

for source_name in os.listdir(co_label):
    if source_name.endswith('.png'):
        # Modify the file name as needed
        new_name = source_name.replace('_co', '_rndwib')
        shutil.copy(os.path.join(co_label, source_name), os.path.join(rndwib_label, new_name))

/tmp/ipykernel_23729/2637588142.py:26: RuntimeWarning: invalid value encountered in cast
  ndwi = np.round(ndwi).astype(np.uint8)


## Generate 'RGB + Green + NIR + NDWI' (6 bands) images & masks
- This cell generate 'RGB + Green + NIR + NDWI' (6 bands) images from the **images/co** and **images/ir** folders & saves them in **images/cognirndwi** folder
- This cell generates a label mask named **labels/masks_cognirndwi** for 'RGB + Green + NIR + NDWI' images. This label mask is similar to the **labels/masks_co** label mask, with the only difference being the name of each respective folder

In [18]:
# RGB + (Green, NIR, NDWI) _ 6 bands
rgb_dir = './images/co'
nir_dir = './images/ir'

cognirndwi_dir = './images/cognirndwi'
if not os.path.exists(cognirndwi_dir):
    os.makedirs(cognirndwi_dir)

# Get list of file names
file_names = [jpg_f.split('.')[0][:-3] for jpg_f in os.listdir(rgb_dir) if jpg_f.endswith('.jpg')]

for name in file_names:
    # Full path rgb, nir
    rgb_path = os.path.join(rgb_dir, name + '_co.jpg')
    nir_path = os.path.join(nir_dir, name + '_ir.jpg')
    cognirndwi_path = os.path.join(cognirndwi_dir, name + '_cognirndwi.tiff')
    
    # Read the images
    rgb_img = cv2.imread(rgb_path)
    rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
    
    # Construct combination (G, NIR, NDWI)
    green = rgb_img[:, :, 1]
    nir = cv2.imread(nir_path)[:, :, 0]
    gnirndwi_img = GreenNirNDWI(np.stack([green, nir], axis=2))

    # Stack the arrays along the third axis (4 bands)
    combined_arr = np.stack([rgb_img[:, :, 0], rgb_img[:, :, 1], rgb_img[:, :, 2],
                             gnirndwi_img[:, :, 0], gnirndwi_img[:, :, 1], gnirndwi_img[:, :, 2]], axis=2)

    # Save the new image as a .tiff file
    tiff.imwrite(cognirndwi_path, combined_arr)

# Generate train_cognirndwi.txt and val_cognirndwi.txt metadata files & save in images/cognirndwi folder
txt_files = [txt_f.split('/')[-1].split('.')[0] for txt_f in os.listdir(rgb_dir) if txt_f.endswith('.txt')]

for txt_file in txt_files:
    # Full path txt
    txt_name = txt_file.replace('_co', '_cognirndwi')
    txt_cognirndwi = os.path.join(cognirndwi_dir, txt_name + '.txt')
    txt_co = os.path.join(rgb_dir, txt_file + '.txt')
    
    # Read the file
    with open(txt_co, 'r') as file:
        lines = file.readlines()

    # Modify the paths
    modified_lines = []
    for line in lines:
        modified_line = line.replace('_co', '_cognirndwi')
        modified_lines.append(modified_line)

    # Write the modified paths back to the file
    with open(txt_cognirndwi, 'w') as file:
        file.writelines(modified_lines)

# LABELS
co_label = './labels/masks_co'
gnirndwi_label = './labels/masks_cognirndwi'
if not os.path.exists(gnirndwi_label):
    os.makedirs(gnirndwi_label)

for source_name in os.listdir(co_label):
    if source_name.endswith('.png'):
        # Modify the file name as needed
        new_name = source_name.replace('_co', '_cognirndwi')
        shutil.copy(os.path.join(co_label, source_name), os.path.join(gnirndwi_label, new_name))

/tmp/ipykernel_23729/1353729320.py:17: RuntimeWarning: invalid value encountered in cast
  ndwi = np.round(ndwi).astype(np.uint8)


## Test

In [10]:
import tifffile as tiff

# Define the path to the .tiff image
image_path = './images/condwi/DJI_0020_103_condwi.tiff'

# Read the image
image = tiff.imread(image_path)
print(image.shape)

(1300, 1600, 4)
